# SQL Workshop
### <font color=indigo>LIVE DEMOs</font>
---

## Introductions


## SQLite & Python

## Using Pandas with SQL

### A Querying Template

---
## Activity: Workbook I
---

## Review Activity I: Filtering

## JOINs

---
## Activity: Workbook 2
---

## Review Activity II: Linking Tables

## Sub-Queries

---
## Activity: Workbook 3
---